In [1]:
import pandas as pd

#df1: esg data --> score & level data of companies
df1 = pd.read_csv('datasets/esg_data.csv')
print(df1.head())
print(df1.info())

#df2: risk ratings of US companies based on esg scores
df2 = pd.read_csv('datasets/SP_500_ESG_Risk_Ratings.csv')
print(df2.head())
print(df2.info())





  ticker                           name currency  \
0    dis                 Walt Disney Co      USD   
1     gm              General Motors Co      USD   
2    gww                WW Grainger Inc      USD   
3    mhk          Mohawk Industries Inc      USD   
4    lyv  Live Nation Entertainment Inc      USD   

                        exchange                            industry  \
0  NEW YORK STOCK EXCHANGE, INC.                               Media   
1  NEW YORK STOCK EXCHANGE, INC.                         Automobiles   
2  NEW YORK STOCK EXCHANGE, INC.  Trading Companies and Distributors   
3  NEW YORK STOCK EXCHANGE, INC.                   Consumer products   
4  NEW YORK STOCK EXCHANGE, INC.                               Media   

                                                logo  \
0  https://static.finnhub.io/logo/ef50b4a2b263c84...   
1  https://static.finnhub.io/logo/9253db78-80c9-1...   
2  https://static.finnhub.io/logo/f153dcda-80eb-1...   
3  https://static.finnhub.io/l

In [2]:
#Data Summary Statistics for df1

import pandas as pd

def compute_summary_statistics(df):
    # Select numerical and categorical columns
    num_cols = df1.select_dtypes(include=['int64', 'float64']).columns
    # Compute summary statistics
    num_summary = df[num_cols].describe().T  # Transpose for better readability
    # Display results
    print("Numerical Columns Summary:")
    print(num_summary)
   

compute_summary_statistics(df1)



Numerical Columns Summary:
                   count           mean            std     min        25%  \
environment_score  722.0     404.806094     145.103870   200.0     240.00   
social_score       722.0     292.182825      57.017406   160.0     243.00   
governance_score   722.0     278.761773      47.031536    75.0     235.00   
total_score        722.0     975.750693     218.751796   600.0     763.00   
cik                722.0  989792.497230  581681.496480  1800.0  723157.25   

                         50%         75%        max  
environment_score      483.0      518.75      719.0  
social_score           302.0      322.75      667.0  
governance_score       300.0      310.00      475.0  
total_score           1046.0     1144.00     1536.0  
cik                1046189.0  1470094.25  1914023.0  


In [3]:
#EDA on df1
import plotly.express as px
import matplotlib as plt
import pandas as pd

industry_counts = df1['total_level'].value_counts().reset_index()
industry_counts.columns = ['total_level', 'count']
fig = px.pie(industry_counts, names='total_level', values='count', title='Distribution of Total ESG Level')
fig.show()


In [4]:
import plotly.express as px

# Scatter plot: total_score vs total_grade, with point size following the total_score
import plotly.express as px

# Create a faceted histogram of total_score by total_grade
fig = px.histogram(df1, x='total_score', color='total_grade', 
                   title='Distribution of Total Scores by Grade',
                   labels={'total_score': 'Total Score', 'total_grade': 'Total Grade'},
                   nbins=20, 
                   )  

fig.update_layout(
    plot_bgcolor = 'rgb(245, 245, 245)',
    yaxis_title = 'Frequency',
    legend=dict(
        orientation='h',  # Horizontal legend
        yanchor='bottom',  # Anchor legend to the bottom
        y=1.05,  # Position it slightly above the plot area
        xanchor='center',  # Center the legend horizontally
        x=0.5,  # Center the legend horizontally
        title = 'Grade'
    )
)

fig.update_traces(marker=dict(line=dict(color='black', width=0.5)))  

# Show the plot
fig.show()


In [12]:
#Clean 2nd Dataset First
import plotly.express as px
import plotly.graph_objects as go

df2_cleaned = df2.dropna(subset = ['Sector', 'Total ESG Risk score'])
print(df2_cleaned.info())


def inspect_data(data):
    return pd.DataFrame(
        {
        "column": data.columns,
        "dtype": data.dtypes.values,
        "instances": data.count().values,
        "unique": data.nunique().values,
        "sum_null": data.isnull().sum().values,
        }
    )
df2_cleaned = df2_cleaned.fillna("NA")

df2_info = inspect_data(df2_cleaned)
print(df2_info)



# from dash import Dash, dcc, html
# import dash_ag_grid as dag

# app = Dash(__name__)



# app.layout = html.Div([
#     dag.AgGrid(
#         id="table",
#         rowData=df.to_dict("records"),
#         columnDefs=column_defs,
#     )
# ])

# if __name__ == "__main__":
#     app.run_server(debug=True)



<class 'pandas.core.frame.DataFrame'>
Index: 430 entries, 1 to 502
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Symbol                  430 non-null    object 
 1   Name                    430 non-null    object 
 2   Address                 430 non-null    object 
 3   Sector                  430 non-null    object 
 4   Industry                430 non-null    object 
 5   Full Time Employees     426 non-null    object 
 6   Description             430 non-null    object 
 7   Total ESG Risk score    430 non-null    float64
 8   Environment Risk Score  430 non-null    float64
 9   Governance Risk Score   430 non-null    float64
 10  Social Risk Score       430 non-null    float64
 11  Controversy Level       430 non-null    object 
 12  Controversy Score       403 non-null    float64
 13  ESG Risk Percentile     430 non-null    object 
 14  ESG Risk Level          430 non-null    object 

In [130]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from scipy.stats import gaussian_kde
from plotly.subplots import make_subplots


esg_scores = df2_cleaned[['Total ESG Risk score', 'Environment Risk Score', 'Governance Risk Score', 'Social Risk Score', 'ESG Risk Level']]

# Melt the dataframe
esg_long = esg_scores.melt(id_vars=["ESG Risk Level"], var_name="ESG Category", value_name="Score")
esg_long["ESG Category"] = esg_long["ESG Category"].astype("category")

# Define category colors
category_colors = {
    "Total ESG Risk score": "#89ac46", 
    "Environment Risk Score": "#d3e671", 
    "Governance Risk Score": "#f8ed8c", 
    "Social Risk Score": "#ff8989"
}

def plot1():

    # Create individual histograms for each ESG category
    for col in esg_scores.columns[:-1]:  
        
        data = esg_long[esg_long['ESG Category'] == col]['Score']
        
        # Create a histogram using Plotly Express
        fig = px.histogram(
            data, 
            x="Score", 
            nbins=25, 
            opacity=0.7, 
            title=f"Distribution of {col}", 
            color_discrete_sequence=[category_colors[col]],  # Use category-specific color
            labels={"Score": "ESG Score", "percent": "Percentage of Scores"}  # Improve axis labels
        )
        
        # Remove gridlines
        fig.update_yaxes(showgrid=False, showticklabels=True, title_text="Percentage of Scores")
        fig.update_xaxes(showgrid=False, title_text="ESG Score")
        
        # Adjust layout settings
        fig.update_layout(
            height=500,
            width=800,
            showlegend=False,
            title_font_size=20,  
            title_x=0.5,
            font=dict(family="Arial", size=12, color="black") 
        )
        
        # Calculate KDE using scipy
        kde = gaussian_kde(data)
        x = np.linspace(data.min(), data.max(), 1000)  
        y = kde(x)  # Calculate KDE values
        
        # Add a secondary y-axis for the KDE line
        fig.update_layout(
            yaxis2=dict(
                title="Density",
                overlaying="y", 
                side="right",  # Place the secondary y-axis on the right
                showgrid=False
            )
        )
        
        # Add a density line (KDE) to the histogram on the secondary y-axis
        fig.add_trace(
            go.Scatter(
                x=x,
                y=y,
                mode='lines',
                name=f'{col} KDE',  # KDE label
                line=dict(color=category_colors[col], dash='solid', width=2),
                showlegend=False,
                yaxis="y2"  
            )
        )
        
        # Add mean and median lines
        mean = data.mean()
        median = data.median()
        
        # Add mean line
        fig.add_vline(
            x=mean, 
            line=dict(color="red", width=2, dash="dash"), 
            annotation_text=f"Mean: {mean:.2f}", 
            annotation_position="top right"
        )
        
        # Add median line
        fig.add_vline(

            x=median, 
            line=dict(color="blue", width=2, dash="dot"), 
            annotation_text=f"Median: {median:.2f}", 
            annotation_position="bottom left"
        )

        fig.update_traces(marker=dict(line=dict(color='grey', width=0.5)))  
        
        # Show plot
        fig.show()

def plot2(): 

    # Create a subplot with 1 row and 4 columns (for 4 categories)
    fig = make_subplots(
        rows=1, cols=4,  
        shared_yaxes=False,  
        column_titles=["Total ESG Risk score", "Environment Risk Score", "Governance Risk Score", "Social Risk Score"]
    )

    # Add histograms and KDE lines to each subplot
    for i, col in enumerate(esg_scores.columns[:-1]):
        esg_category_data = esg_long[esg_long['ESG Category'] == col]
        
        # Create the histogram for each category
        fig.add_trace(
            go.Histogram(
                x=esg_category_data['Score'],
                nbinsx=25,
                opacity=0.7,
                histnorm="percent",
                marker_color=category_colors[col],
                name=f"{col} Histogram", 
                showlegend = False
            ),
            row=1, col=i+1  # Place in the first row, i-th column
        )

        # Calculate KDE using scipy gaussian_kde
        kde = gaussian_kde(esg_category_data['Score'].dropna())  # KDE calculation (remove NaNs)
        x = np.linspace(esg_category_data['Score'].min(), esg_category_data['Score'].max(), 1000)  # X values for KDE line
        y = kde(x)  # KDE values

        # Add a KDE line to the plot
        fig.add_trace(
            go.Scatter(
                x=x,
                y=y * 100,  # Scale the KDE to match percentage values of histogram
                mode="lines",
                name=f"{col} KDE",
                line=dict(color=category_colors[col], width=2),
                showlegend=False  # Prevent adding to legend
            ),
            row=1, col=i+1  # Place in the first row, i-th column
        )

    # Update layout
    fig.update_layout(
        height=600,
        width=1200,
        showlegend=True,  # Optionally show legend for the color mapping
        title="Distribution of ESG Scores with KDE Lines",
        title_x=0.5,  # Center the title
        title_y=0.98,  # Adjust the title positioning
        bargap=0.1
    )

    # Remove gridlines and adjust axes
    fig.update_yaxes(showgrid=False)
    fig.update_xaxes(showgrid=False)

    # Show plot
    fig.show()

plot1()
plot2()



In [166]:
#bar charts of risk and controvery levels
levels = df2_cleaned[['Controversy Level', 'ESG Risk Level', 'Sector']]

# Remove "Controversy Level=" from values in the column
levels["Controversy Level"] = levels["Controversy Level"].str.replace("Controversy Level", "", regex=False)

df_controversy_lvl = levels.groupby(["Sector", "Controversy Level"]).size().reset_index(name="Count")
df_controversy_lvl["Proportion"] = df_controversy_lvl.groupby("Sector")["Count"].transform(lambda x: x / x.sum())
df_controversy_lvl["Controversy Level"] = (df_controversy_lvl["Controversy Level"].astype("category")).str.replace("Controversy Level", "", regex=False).str.strip()

df_esg_lvl = levels.groupby(["Sector", "ESG Risk Level"]).size().reset_index(name="Count")
df_esg_lvl["Proportion"] = df_esg_lvl.groupby("Sector")["Count"].transform(lambda x: x / x.sum())
df_esg_lvl["ESG Risk Level"] = (df_esg_lvl["ESG Risk Level"].astype("category")).str.strip()


custom_colors = {
    "High": "#f78e31",
    "Medium": "#fbb040",
    "Low": "#208b3a",
    "Moderate": "#cbdb47",
    "Medium" :"#cbdb47",
    "Severe": "#e83715",
    "Significant":"#f26b21",
    "None": "#fcec52",
    "Negligible": "#fcec52"
}



fig1 = px.bar(
    df_controversy_lvl, 
    x="Sector", 
    y="Proportion",  # Use proportion instead of count
    color="Controversy Level", 
    title="Composition of Controversy Levels by Sector",
    barmode="stack",
    text=df_controversy_lvl["Proportion"].apply(lambda x: f"{x:.0%}"),
    color_discrete_map=custom_colors,
    category_orders={"Controversy Level": ["Severe", "Significant", "High", "Moderate", "Low", "None"]}
)

fig1.update_layout(
    width=1000,  
    height=600, 
    title_x=0.5, 
    title_y=0.9,
    margin=dict(l=100, r=100, t=100, b=100),  
    legend_title_text="Controversy Level"  
)

# Format y-axis as percentage
fig1.update_yaxes(showgrid = False, tickformat=".0%", title="Percentage of Companies")

# Show plot
fig1.show()


fig2 = px.bar(
    df_esg_lvl, 
    x="Sector", 
    y="Proportion",  # Use proportion instead of count
    color="ESG Risk Level", 
    title="Composition of ESG Risk Levels by Sector",
    barmode="stack",
    text=df_esg_lvl["Proportion"].apply(lambda x: f"{x:.0%}"),
    color_discrete_map=custom_colors,
    category_orders={"ESG Risk Level": ["Severe", "High", "Medium", "Low", "Negligible"]}
)

fig2.update_layout(
    width=1000,  
    height=600, 
    title_x=0.5, 
    title_y=0.9,
    margin=dict(l=100, r=100, t=100, b=100),  
    legend_title_text="ESG Risk Level"  
)

# Format y-axis as percentage
fig2.update_yaxes(showgrid = False, tickformat=".0%", title="Percentage of Companies (%)")

fig2.show()




    
    



C:\Users\alyss\AppData\Local\Temp\ipykernel_8800\3660596521.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#Scatter plot of company size: number of employees and esg risk percentile 
#compared next to the distribution of industry and risk percentile bar charts?
data = df2_cleaned[["Full Time Employees", "ESG Risk Percentile"]]



#esg score and 


